# Уровень заработной платы

#### **Информация о данных**

Используемый набор данных является выборкой из базы данных переписи населения 1994 года, сделанной Барри Беккером. Датасет содержит следующие признаки:

1. **Возраст** (`age`): отражает возраст респондента (от 17 до 90 лет).
2. **Рабочий класс** (`workclass`): отражает статус занятости респондента.
3. **Оценка уникальности** (`fnlwgt`): отражает примерную оценку количества людей, которое представляет каждая строка данных.
4. **Образование** (`education`): отражает уровень образования респондента.
5. **Ступень образования** (`education-num`): отражает статус образования от дошкольного до докторантуры (от 1 до 16).
6. **Семейное положение** (`marital-status`): отражает семейное положение респондента.
7. **Сфера занятости** (`occupation`): отражает сферу занятости респондента.
8. **Семейная роль** (`relationship`): отражает роль в семье респондента.
9. **Раса** (`race`): отражает расовую принадлежность респондента.
10. **Пол** (`sex`): отражает пол респондента (мужчина, женщина).
11. **Прирост капитала** (`capital-gain`): отражает увеличение капитала у респондента.
12. **Потеря капитала** (`capital-loss`): отражает уменьшение капитала у респондента.
13. **Трудоспособность** (`hours-per-week`): отражает количество отработанных часов за неделю (от 1 до 99).
14. **Страна рождения** (`native-country`): отражает страну, в которой родился респондент.
15. **Зарплата** (`salary`): отражает уровень заработной платы респондента (<=50K, >50K).

#### **Описание задачи**

*Задача заключается в прогнозировании уровеня заработной платы.*

В данной работе требуется проанализировать исходные данные, выяснить какие признаки оказывают наиболее сильное влияние на уровень заработной платы, применить различные алгоритмы классификации, которые смогут предсказывать уровень заработной платы по входным в данным.

> Источник данных: https://www.kaggle.com/datasets/ayessa/salary-prediction-classification


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

In [3]:
url = r"C:\Users\79960\МашинноеОбучение_ннгу_2023\ML_course\salary.csv"
data_raw = pd.read_csv(url)
data_raw.head(10)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


In [4]:
dr = data_raw.copy()
dr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  salary          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [5]:
dr.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
salary            0
dtype: int64

*В таблице отсутствуют пропущенные данные числовых признаков!*

In [6]:
dr.shape

(32561, 15)

Для анализа имеем **32561 объекта** и **15 признаков**. Посмотрим на *числовые* и *категориальные* признаки по отдельности.

### **2.1. Числовые признаки**

In [7]:
numeric_parameters = [column for column in dr.columns if dr[column].dtype.name != 'object']
numeric_parameters

['age',
 'fnlwgt',
 'education-num',
 'capital-gain',
 'capital-loss',
 'hours-per-week']

In [8]:
data = dr[numeric_parameters]

Имеем **6** числовых признаков. Посмотрим их основные характеристики

In [9]:
dr.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


В таблице представлены основые статистические характеристики:

* Количество (`count`)
* Медиана (`mean`)
* Среднеквадратическое отклонение (`std`)
* Минимум (`min`)
* Квантиль 25%
* Квантиль 50% 
* Квантиль 75% 
* Максимум (`max`)

На данном этапе можно заметить странное распределение в признаках *capital-gain* и *capital-loss*. Скорее всего это связано с тем, что данные признаки равняются нулю для большиства опрошенных. Все остальные характеристики выглядят достаточно правдоподобно.


### **2.2. Категориальные признаки**

Выделим среди всех признаков — категориальные

In [10]:
category_parameters = [column for column in dr.columns if dr[column].dtype.name == 'object']
category_parameters

['workclass',
 'education',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native-country',
 'salary']

Имеем **9** категориальных признаков. Рассмотрим распределение каждого из данных признаков.

In [ ]:
for col in [column for column in dr.columns if dr[column].dtype.name == 'object']:
    print(f"{col} DISTRIBUTION")
    print(dr[col].value_counts())
    print()

Просматривая полученные значения, можно заметить, что среди категориальных признаков **workclass**, **occupation**, **native-country** имеются пропущенные данные, которые обозначаются **' ?'**. Произведем замену данных значений на NaN (Not a Number).

In [ ]:
dr['workclass'].replace(' ?', np.nan, inplace=True)
dr['occupation'].replace(' ?', np.nan, inplace=True)
dr['native-country'].replace(' ?', np.nan, inplace=True)

##  **3. Борьба с пропущенными значениями**

Из анализа данных, содержащихся в категориальных признаках, обнаружилось, что в датасете присутствуют пропуски, с которыми нужно как-то побороться. Рассмотрим какие признаки имеют пропущенные значения:

In [ ]:
dr.isnull().sum()

Существуют различные способы борьбы с пропусками. Самый примитивный — просто удалить те строки, в которых встречаются пропущенные значения. Так как датасет содержит не особо большое количество строк, то удаление пропущенных данных может привести к неточности дальнейшего исследования, поэтому способом заполнения пропусков будет *поиск наиболее популярных значений среди данного признака.* Посмотрим на часть датасета с пропусками:

In [ ]:
dr_with_nan = dr[dr['workclass'].isnull()]
dr_with_nan.head(10)

В данной выборке можно разглядеть, что везде, где отсутствует признак workclass, также отсутствует и признак occupation. Возможно между ними имеется некая связь. Поэтому найдем наиболее популярное значение признака **workclass** и затем по найденному значению выберем самое популярное среди **occupation**. Значения для признака **native-country**	заполним аналогично — наиболее популярным.

Найдем самое популярное значение среди признака **workclass**

In [ ]:
dr['workclass'].mode()[0]

Найдем самое популярное значение среди признака **occupation**, учитывая что *workclass* = *'Private'*

In [ ]:
dr['occupation'][dr['workclass'] == dr['workclass'].mode()[0]].mode()[0]

Найдем самое популярное значение среди признака **native-country**

In [ ]:
dr['native-country'].mode()[0]

Заменим соответствующие пропуски найденными значениями

In [ ]:
dr['workclass'].fillna(dr['workclass'].mode()[0], inplace=True)
dr['occupation'].fillna(dr['occupation'][dr['workclass'] == dr['workclass'].mode()[0]].mode()[0], inplace=True)
dr['native-country'].fillna(dr['native-country'].mode()[0], inplace=True)

In [ ]:
dr.isnull().sum()

Мы успешно поборолись с пропущенными значениями, можно переходить к дальнейшему исследованию.

##  **4. Визуализация данных**

In [ ]:
dr.info()

*Простроим графическое представление матрицы корреляции*

In [ ]:
plt.figure(figsize = (8,6))
sb.heatmap(data.corr(), annot=True, cmap='Blues')
plt.show()

Как результат, признаки практически не коррелируют друг с другом.

In [ ]:
color = ["#3498db","#34495e"]
sb.set_palette(color)

*Посмотрим на какой возраст приходится наибольший заработок*

In [ ]:
plt.figure(figsize=(12, 8))
sb.boxplot(data=dr, y='age', x='salary')
plt.show()

Можно увидеть, что среди опрошенных, у которых зарплата более 50К, преобладают люди в возрасте примерно от 35 до 50. Зарплату менее 50К получают люди примерно от 25 до 45.

*Посмотрим зависимость заработной платы от характеристики, которая обозначает количество людей со схожими признаками*

In [ ]:
plt.figure(figsize=(12, 8))
sb.boxplot(data=dr, y='fnlwgt', x='salary')
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
sb.kdeplot(data=dr, x='fnlwgt', hue='salary')
plt.show()

Трудно разглядеть зависимость зарплаты от данного признака. Попробуем отбросить его при дальнейшей обработке данных.

*Посмотрим распределение заработной платы по принадлежности к рабочему классу*

In [ ]:
plt.figure(figsize=(12, 8))
sb.countplot(data=dr, x='workclass', hue='salary')
plt.show()

Среди людей рабочего класса можем наблюдать наибольшее количество людей, которые получают зарплату более 50К, кроме того данный класс преобладает и среди тех, кто пролучает зарплату менее 50К. 

*Посмотрим распределение заработной платы среди уровня образования*

In [ ]:
plt.figure(figsize=(12, 8))
sb.countplot(data=dr, y='education', hue='salary')
plt.show()

Очевидно, что уровень образования играет ключевую роль в формировании заработной платы. По графику видно, что большинство тех, кто получает более 50К имеют образование начиная от высшей школы или колледжа и заканчивая магистратурой.

*Посмотрим зависимость заработной платы от семейного положения*

In [ ]:
plt.figure(figsize=(12, 8))
sb.countplot(data=dr, x='marital-status', hue='salary')
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
sb.countplot(data=dr, x='relationship', hue='salary')
plt.show()

*Посмотрим зависимость заработной платы от расы*

In [ ]:
plt.figure(figsize=(12, 8))
sb.countplot(data=dr, x='race', hue='salary')
plt.show()

*Посмотрим зависимость заработной платы от пола*

In [ ]:
plt.figure(figsize=(12, 8))

sb.countplot(data=dr, x='sex', hue='salary')
plt.show()

Высокий заработок преобладает среди мужчин.

In [ ]:
plt.figure(figsize=(12, 8))
sb.kdeplot(data=dr, x='capital-gain', hue='salary')
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
sb.kdeplot(data=dr, x='capital-loss', hue='salary')
plt.show()

По графикам можно увидеть, что данные характеристики не оказывают никакого влияния на уровень заработной платы, скорее всего потому что для большинства объектов являются нулём, и не смогут быть полезны при обучении модели. В дальнейшем мы удалим эти признаки.

*Посмотрим зависимость заработной платы от количества отработанных часов в неделю*

In [ ]:
plt.figure(figsize=(12, 8))
sb.boxplot(data=dr, x='salary', y='hours-per-week')
plt.show()

По графикам видно, что работники, которые трудятся более 40 часов в неделю получают зарплату выше, чем те, которые работают менее 40 часов.

##  **5. Обработка данных**

#### **5.1. Обработка числовых признаков**